## Ripple Adder from Toggle Flip-Flops

In this example we create a ripple adder from toggle flip-flops. 
We also show how to define new `Magma` `Circuits`
and introduce *generators*.

In [1]:
import magma as m

import os
os.environ['MANTLE_TARGET'] = 'ice40'
from mantle import DFF

import mantle lattice ice40
import mantle lattice mantle40


In the last example, we defined a function that created a
toggle flip-flop (TFF) from a DFF and an XOR gate.
Let's convert the TFF to a `Circuit`. 
In `Magma` a `Circuit` is equivalent to a verilog *module*.
Circuits can be instanced and then wired to other circuits.

In [2]:
class TFF(m.Circuit):
    IO = ["O", m.Out(m.Bit)] + m.ClockInterface()
    @classmethod
    def definition(io):
        ff = DFF()
        m.wire( ff(~ff.O), io.O )
        m.wireclock( io, ff )

In [3]:
def DefineRippleCounter(n):
    class RippleCounter(m.Circuit):
        name = 'Ripple' + str(n)
        IO = ["O", m.Out(m.Bits(n))] + m.ClockInterface()
        @classmethod
        def definition(io):
            tffs = [TFF() for i in range(n)]
            O = io.CLK
            for i in range(n):
                m.wire(O, tffs[i].CLK)
                O = tffs[i].O
                m.wire(O, io.O[i])
    return RippleCounter

In [4]:
N = 4
Ripple = DefineRippleCounter(N)

In [5]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.J1[0].rename('J1').input().on()  # rename so J1 is not an array
for i in range(N):
    icestick.J3[i].output().on()

main = icestick.DefineMain()
ripple = Ripple()
m.wire( ripple.O, main.J3 )
m.EndDefine()

## Compile and Build

In [6]:
m.compile("build/ripple", main)

compiling TFF
compiling Ripple4
compiling main


In [7]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ripple.blif' ripple.v
arachne-pnr -q -d 1k -o ripple.txt -p ripple.pcf ripple.blif 
icepack ripple.txt ripple.bin
# iceprog ripple.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build
